In [1]:
from array import *

import pandas as pd
df = pd.read_csv('data/imitation+privacy-policy.csv', header=None)
print(df)
# print(df.loc[[0]][1])

    0   1   2   3   4   5   6   7   8   9   ...  22  23  24  25  26  27  28  \
0    4   2   4   2   2   1   3   2   1   2  ...   1   2   4   2   4   1   3   
1    4   1   3   1   3   1   1   2   3   2  ...   2   2   4   1   4   1   1   
2    4   2   4   2   1   2   3   1   3   1  ...   3   2   4   2   4   1   1   
3    4   1   4   2   1   2   1   1   1   2  ...   1   2   1   2   3   2   1   
4    4   2   4   2   2   2   3   1   1   2  ...   1   2   4   2   4   1   1   
5    4   1   4   1   1   2   3   1   1   2  ...   1   2   3   1   4   2   2   
6    2   1   1   1   1   2   1   2   3   1  ...   1   1   1   2   3   1   1   
7    1   2   1   1   1   2   1   1   1   1  ...   3   1   2   1   2   1   1   
8    4   2   4   2   1   2   3   2   1   2  ...   3   2   1   2   3   2   1   
9    4   2   4   1   1   2   3   1   1   2  ...   4   2   4   2   4   2   1   
10   4   1   4   2   2   2   3   2   4   1  ...   1   2   3   1   4   2   3   
11   4   2   3   1   3   2   2   1   1   2  ...   1 

In [2]:
import numpy as np
df_adj = pd.read_csv('data/adjacency.csv', header=None, index_col=False)
#df_adj = pd.DataFrame(np.loadtxt('data/adjacency.csv', dtype=int))
#df_adj

In [3]:
import random

agentlocations = []
agentpolicies = []

initlocation = []
initpolicy = []

column_names = ['step', 'id', 'context', 'policy']
df_context_policies  = pd.DataFrame(columns = column_names)
df_context_policies

for a in range(0,24):
    #print(a)
    context = random.randint(1,16)
    #initlocation.insert(a, location)
    #print(int(df.loc[[a]][initlocation[a]*2-2]))
    policy = int(df.loc[[a]][context*2-2])
    #initpolicy.insert(a, int(df.loc[[a]][location*2-2]))
    #currentlocations.append(location)
    #currentpolicies.append(int(df.loc[[a]][location*2-2]))
    
    df_context_policies.loc[len(df_context_policies)] = [0, a, context, policy]
    
#print(initlocation)    
#print(initpolicy)    

#agentlocations.append(initlocation)
#agentpolicies.append(initpolicy)



for t in range(1,100):
    #currentlocations = agentlocations[t-1]
    #currentpolicies = agentpolicies[t-1]
    for a in range(0,24):
        moveprob = random.randint(1,100)
        if moveprob < 25:
            context = random.randint(1,16)
            policy = int(df.loc[[a]][context*2-2])
        else:
            index = a + (t-1)*24
            context = df_context_policies.loc[index]['context']
            #context = df_context_policies.loc[(df_context_policies['step'] == t-1) & (df_context_policies['id'] == a)]['context']
            policy = df_context_policies.loc[index]['policy']
            #policy = df_context_policies.loc[(df_context_policies['step'] == t-1) & (df_context_policies['id'] == a)]['policy']

        df_context_policies.loc[len(df_context_policies)] = [t, a, context, policy]
            
        #currentsharingpolicies[]

    #print("\n Current:", currentlocations)
    #agentlocations.append(currentlocations)
    #print("\n All: ", agentlocations)
    
    #agentpolicies.append(currentpolicies)

# #print("Locations", agentlocations)
# #print("Policies", agentpolicies)



# df_locations=pd.DataFrame(data=agentlocations)
# #print("\n\n", agentlocations)
# #print(df_locations)

df_context_policies

,step,id,context,policy
0,0,0,13,4
1,0,1,2,3
2,0,2,2,4
3,0,3,6,1
4,0,4,16,1
5,0,5,15,2
6,0,6,4,1
7,0,7,4,1
8,0,8,15,1
9,0,9,13,4


In [4]:
import matplotlib.pyplot as plt  
#df_context_policies.shape
#df_context_policies.head

#X = df_context_policies.drop(['policy', 'step', 'id'], axis=1)  
X = df_context_policies.drop(['policy', 'step'], axis=1)  
y = df_context_policies['policy']

print(X)
print(y)

      id context
0      0      13
1      1       2
2      2       2
3      3       6
4      4      16
5      5      15
6      6       4
7      7       4
8      8      15
9      9      13
10    10      16
11    11       7
12    12       7
13    13      10
14    14      12
15    15       9
16    16       7
17    17       9
18    18      13
19    19       1
20    20      16
21    21       8
22    22      11
23    23       4
24     0      12
25     1       2
26     2       2
27     3       6
28     4      16
29     5       5
...   ..     ...
2370  18      11
2371  19      13
2372  20       1
2373  21       6
2374  22      10
2375  23       4
2376   0      12
2377   1       4
2378   2      12
2379   3       4
2380   4       6
2381   5      12
2382   6       4
2383   7       8
2384   8       7
2385   9      10
2386  10       9
2387  11       5
2388  12      13
2389  13      11
2390  14      10
2391  15       4
2392  16      12
2393  17      12
2394  18      11
2395  19      13
2396  20      

In [5]:
from sklearn.model_selection import train_test_split  
y=y.astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20) 


print(X_train, y_train)

      id context
2123  11       1
2104  16      12
818    2       2
1362  18      15
1511  23      12
1607  23       5
1529  17       4
1862  14       3
1752   0      15
1787  11      10
998   14       8
2300  20       5
2337   9       6
2037  21      11
1390  22       5
152    8      13
1976   8       9
2274  18       5
135   15       1
1617   9       6
461    5       1
1764  12       6
2097   9       5
895    7      14
922   10      13
1419   3      16
182   14       2
753    9       3
1656   0      15
1926   6      16
...   ..     ...
1075  19       8
1310  14       1
1690  10      12
328   16       7
1393   1       7
102    6       4
558    6      14
2355   3       1
277   13       8
600    0       2
2205  21       4
236   20      15
325   13       3
1256   8       2
437    5       1
335   23       9
1635   3       9
1244  20       1
2079  15      14
388    4       1
974   14      12
382   22      11
281   17       2
2090   2      12
939    3      10
516   12      15
1467   3      

In [6]:
from sklearn.svm import SVC  
svclassifier = SVC(kernel='linear')  
svclassifier.fit(X_train, y_train)  

y_pred = svclassifier.predict(X_test)  


from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  

[[217   0   0   0]
 [ 53   0   0   0]
 [ 98   0   0   0]
 [112   0   0   0]]
              precision    recall  f1-score   support

           1       0.45      1.00      0.62       217
           2       0.00      0.00      0.00        53
           3       0.00      0.00      0.00        98
           4       0.00      0.00      0.00       112

    accuracy                           0.45       480
   macro avg       0.11      0.25      0.16       480
weighted avg       0.20      0.45      0.28       480



/home/najmeri/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
d = {'id': [23, 22, 1], 'context': [4, 7, 1]}
df = pd.DataFrame(data=d)
svclassifier.predict(df)

array([1, 1, 4])